<a href="https://colab.research.google.com/github/daothuphuong98/COVID19-NER/blob/main/XLMR_BASE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/VinAIResearch/PhoNER_COVID19.git

Cloning into 'PhoNER_COVID19'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 58 (delta 23), reused 41 (delta 18), pack-reused 0
Unpacking objects: 100% (58/58), done.


In [ ]:
!pip install datasets pytorch-crf==0.7.2 seqeval transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 452 kB 6.6 MB/s 
     |████████████████████████████████| 43 kB 1.1 MB/s 
     |████████████████████████████████| 5.8 MB 25.1 MB/s 
     |████████████████████████████████| 212 kB 56.3 MB/s 
     |████████████████████████████████| 182 kB 50.8 MB/s 
     |████████████████████████████████| 132 kB 82.4 MB/s 
     |████████████████████████████████| 127 kB 73.3 MB/s 
     |████████████████████████████████| 7.6 MB 71.5 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16179 sha256=4a8ca8a9da563a839645d32dc1dabfcf6bd71304934ad8e5d44c6f347c105523
  Stored in directory: /root/.cache/pip/wheels/ad/5c/ba/05fa33fa5855777b7d686e843ec07452f22a66a138e290e732
Successfully built seqeval
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.

In [ ]:
import pandas as pd
import numpy as np
import re
import torch
import tensorflow as tf
import torch.nn as nn
from torchcrf import CRF
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import BertForTokenClassification, AdamW, XLMRobertaTokenizer, XLMRobertaConfig, XLMRobertaModel, TrainingArguments, AutoTokenizer, AutoConfig, Trainer, AutoModel, DataCollatorForTokenClassification
from tqdm import tqdm, trange, tqdm_notebook
from transformers.modeling_outputs import TokenClassifierOutput
from transformers.models.roberta.modeling_roberta import RobertaModel
from transformers.models.roberta.modeling_roberta import RobertaPreTrainedModel
from seqeval.metrics import f1_score, classification_report
from datasets import ClassLabel, Dataset, DatasetDict
# from transformers import get_linear_schedule_with_warmup, get_constant_schedule_with_warmup, get_cosine_with_hard_restarts_schedule_with_warmup
pd.set_option('display.max_colwidth', -1)

<ipython-input-1-f82015fadac9>:17: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print(device)

MAX_LEN = 256
BS = 32

linear_dropout = 0.3
bert_att_dropout = 0.15
bert_hidd_dropout = 0.2

bert_lr = 1e-5
bert_weight_decay = 1e-5
softmax_lr = 1e-3
softmax_weight_decay = 1e-3

cuda


In [ ]:
!nvidia-smi

Tue Dec 20 11:23:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0    31W /  70W |      3MiB / 15109MiB |      5%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#Data

In [ ]:
def handle_file(link):
    with open(link) as f:
        contents = f.readlines()
    idx_sentence, idx_arr, word, tag = 0, [], [], []
    for line in contents:
        arr_info = line.split(' ')
        # print(arr_info)
        if len(arr_info) == 1:
            idx_sentence += 1
            continue
        word.append(arr_info[0])
        tag.append(arr_info[1].replace('\n', ''))
        idx_arr.append(idx_sentence)
    return pd.DataFrame(
        data = {
            'idx': idx_arr,
            'word': word,
            'tag': tag
        }
    )

In [ ]:
word_train_data = handle_file('/content/PhoNER_COVID19/data/word/train_word.conll')
word_test_data = handle_file('/content/PhoNER_COVID19/data/word/test_word.conll')
word_val_data = handle_file('/content/PhoNER_COVID19/data/word/dev_word.conll')

word_train_data['word_process'] = word_train_data['word'].apply(lambda x: re.sub('[0-9]+', "<NUM>", x).lower())
word_test_data['word_process'] = word_test_data['word'].apply(lambda x: re.sub('[0-9]+', "<NUM>", x).lower())
word_val_data['word_process'] = word_val_data['word'].apply(lambda x: re.sub('[0-9]+', "<NUM>", x).lower())

filter_string = '?!"/\{}().,'
word_train_data = word_train_data[word_train_data['word_process'].apply(lambda x: x not in filter_string)]
word_test_data = word_test_data[word_test_data['word_process'].apply(lambda x: x not in filter_string)]
word_val_data = word_val_data[word_val_data['word_process'].apply(lambda x: x not in filter_string)]

word_train_data_group = word_train_data.groupby('idx').agg({'word_process': ' '.join, 'word': ' '.join, 'tag': list}) 
word_test_data_group = word_test_data.groupby('idx').agg({'word_process': ' '.join, 'word': ' '.join, 'tag': list}) 
word_val_data_group = word_val_data.groupby('idx').agg({'word_process': ' '.join, 'word': ' '.join, 'tag': list}) 

word_train_data_group['len_tag'] = word_train_data_group['tag'].apply(lambda x: len(x))
word_val_data_group['len_tag'] = word_val_data_group['tag'].apply(lambda x: len(x))
word_test_data_group['len_tag'] = word_test_data_group['tag'].apply(lambda x: len(x))

word_train_data_group['tag_string'] = word_train_data_group['tag'].apply(lambda x: ' '.join(x))
word_val_data_group['tag_string'] = word_val_data_group['tag'].apply(lambda x: ' '.join(x))
word_test_data_group['tag_string'] = word_test_data_group['tag'].apply(lambda x: ' '.join(x))

# train_data_group = train_data_group[train_data_group['len_tag'] <= 55].reset_index()
# val_data_group = val_data_group[val_data_group['len_tag'] <= 55].reset_index()
# test_data_group = test_data_group[test_data_group['len_tag'] <= 55].reset_index()

In [ ]:
word_train_data_group[word_train_data_group['len_tag'] < 15].sample(20)

,word_process,word,tag,len_tag,tag_string
idx,,,,,
161,kết_quả xét_nghiệm mẫu bệnh_phẩm dương_tính với ncov,Kết_quả xét_nghiệm mẫu bệnh_phẩm dương_tính với nCoV,"[O, O, O, O, O, O, O]",7,O O O O O O O
1579,hiện <num> bệnh_nhân được cách_ly điều_trị tại trung_tâm y_tế huyện cam_lâm khánh_hoà,Hiện 2 bệnh_nhân được cách_ly điều_trị tại Trung_tâm Y_tế huyện Cam_Lâm Khánh_Hoà,"[O, O, O, O, O, O, O, B-LOCATION, I-LOCATION, I-LOCATION, I-LOCATION, B-LOCATION]",12,O O O O O O O B-LOCATION I-LOCATION I-LOCATION I-LOCATION B-LOCATION
1258,ngày <num> đến <num> - <num> về nhà ông_bà ngoại ở thị_trấn nam_sách chơi,Ngày 6 đến 7 - 8 về nhà ông_bà ngoại ở thị_trấn Nam_Sách chơi,"[O, B-DATE, O, B-DATE, I-DATE, I-DATE, O, O, O, O, O, B-LOCATION, I-LOCATION, O]",14,O B-DATE O B-DATE I-DATE I-DATE O O O O O B-LOCATION I-LOCATION O
3839,trung_tâm kiểm_soát bệnh_tật hà_nội lấy mẫu xét_nghiệm kết_quả bệnh_nhân dương_tính ncov,Trung_tâm Kiểm_soát bệnh_tật Hà_Nội lấy mẫu xét_nghiệm kết_quả bệnh_nhân dương_tính nCoV,"[B-ORGANIZATION, I-ORGANIZATION, I-ORGANIZATION, I-ORGANIZATION, O, O, O, O, O, O, O]",11,B-ORGANIZATION I-ORGANIZATION I-ORGANIZATION I-ORGANIZATION O O O O O O O
893,<num>h sáng ngày <num>/<num> bệnh_nhân được chuyển đến khoa y_học nhiệt_đới bệnh_viện đà_nẵng,2h sáng ngày 27/7 bệnh_nhân được chuyển đến khoa Y_học nhiệt_đới Bệnh_viện Đà_Nẵng,"[O, O, O, B-DATE, O, O, O, O, B-LOCATION, I-LOCATION, I-LOCATION, I-LOCATION, I-LOCATION]",13,O O O B-DATE O O O O B-LOCATION I-LOCATION I-LOCATION I-LOCATION I-LOCATION
2262,hiện bệnh_nhân điều_trị tại bệnh_viện dã_chiến củ_chi,Hiện bệnh_nhân điều_trị tại Bệnh_viện Dã_chiến Củ_Chi,"[O, O, O, O, B-LOCATION, I-LOCATION, I-LOCATION]",7,O O O O B-LOCATION I-LOCATION I-LOCATION
1535,bệnh_nhân <num> ở xã khải_xuân huyện thanh_ba tỉnh phú_thọ,Bệnh_nhân 994 ở xã Khải_Xuân huyện Thanh_Ba tỉnh Phú_Thọ,"[O, B-PATIENT_ID, O, B-LOCATION, I-LOCATION, B-LOCATION, I-LOCATION, B-LOCATION, I-LOCATION]",9,O B-PATIENT_ID O B-LOCATION I-LOCATION B-LOCATION I-LOCATION B-LOCATION I-LOCATION
4533,vài ngày trước khi có dấu_hiệu sốt bệnh_nhân cùng chồng đến chợ đầu_mối mua_sắm,Vài ngày trước khi có dấu_hiệu sốt bệnh_nhân cùng chồng đến chợ đầu_mối mua_sắm,"[O, O, O, O, O, O, B-SYMPTOM_AND_DISEASE, O, O, O, O, O, O, O]",14,O O O O O O B-SYMPTOM_AND_DISEASE O O O O O O O
2955,đến thời_điểm này không có thêm nhân_viên y_tế nào của bạch_mai mắc bệnh,Đến thời_điểm này không có thêm nhân_viên y_tế nào của Bạch_Mai mắc bệnh,"[O, O, O, O, O, O, O, O, O, O, B-ORGANIZATION, O, O]",13,O O O O O O O O O O B-ORGANIZATION O O


#Model

In [ ]:
pre_train = {
    'mbert': 'bert-base-multilingual-cased',
    'xlmr_base': 'xlm-roberta-base',
    'xlmr_large': 'xlm-roberta-large',
    'phobert': 'vinai/phobert-base'
}
pre_trained_name = pre_train['xlmr_base']
MAX_LEN = 128

In [ ]:
tag_values = ['O',
 'B-ORGANIZATION',
 'I-ORGANIZATION',
 'B-SYMPTOM_AND_DISEASE',
 'I-SYMPTOM_AND_DISEASE',
 'B-LOCATION',
 'B-DATE',
 'B-PATIENT_ID',
 'B-AGE',
 'B-NAME',
 'I-DATE',
 'B-JOB',
 'I-LOCATION',
 'B-TRANSPORTATION',
 'B-GENDER',
 'I-TRANSPORTATION',
 'I-JOB',
 'I-NAME',
 'I-AGE',
 'I-PATIENT_ID']
tags = ClassLabel(num_classes=len(tag_values), names=tag_values)

In [ ]:
index2tag = {idx: tag for idx, tag in enumerate(tags.names)}
tag2index = {tag: idx for idx, tag in enumerate(tags.names)}
tokenizer = AutoTokenizer.from_pretrained(pre_trained_name, do_lower_case=False,use_fast=True)
config = AutoConfig.from_pretrained(pre_trained_name, output_hidden_states=False,
                                    id2label=index2tag, label2id=tag2index)
# model = AutoModel.from_pretrained(pre_trained_name, config=config)
config.hidden_dropout_prob = bert_hidd_dropout
config.attention_probs_dropout_prob = bert_att_dropout

## Without CRF

In [ ]:
def align_predictions(predictions, label_ids):
  preds = np.argmax(predictions, axis=2)
  batch_size, seq_len = preds.shape
  labels_list, preds_list = [], []
  for batch_idx in range(batch_size):
    example_labels, example_preds = [], []
    for seq_idx in range(seq_len):
      if label_ids[batch_idx, seq_idx] != -100:
        example_labels.append(index2tag[label_ids[batch_idx][seq_idx]])
        example_preds.append(index2tag[preds[batch_idx][seq_idx]])
    labels_list.append(example_labels)
    preds_list.append(example_preds)
  return preds_list, labels_list

def compute_metrics(eval_pred):
  print(eval_pred)
  y_pred, y_true = align_predictions(eval_pred.predictions,
                                     eval_pred.label_ids)
  return {"f1": f1_score(y_true, y_pred)}

In [ ]:
def encode(r):
  X = tokenizer(r['word_process'], 
                truncation=True,
                padding='max_length', 
                max_length=MAX_LEN)
  previous_word_idx = None
  label_ids = []
  for word_idx in X.word_ids():
    if word_idx is None or word_idx == previous_word_idx:
      label_ids.append(-100)
    elif word_idx != previous_word_idx:
      label_ids.append(tag2index[r['tag'][word_idx]])
    previous_word_idx = word_idx
  return X.input_ids, label_ids

def preprocess(df):
  df[['tokenized_sentence','tokenized_labels']] = df.apply(lambda x: encode(x),  axis=1, result_type='expand')
  return df

word_train_data_group = preprocess(word_train_data_group)
word_val_data_group = preprocess(word_val_data_group)
word_test_data_group = preprocess(word_test_data_group)

In [ ]:
class XLMRobertaForTokenClassification(RobertaPreTrainedModel):
  config_class = XLMRobertaConfig
  def __init__(self, config, num_layer=197):
    super().__init__(config)
    self.num_labels = config.num_labels
    # Load model body
    self.roberta = RobertaModel(config, add_pooling_layer=False)
    # cnt = 0
    # for param in self.roberta.named_parameters():
    #   cnt += 1
    #   if cnt>=num_layer:
    #       param[1].requires_grad = True
    #   else:
    #       param[1].requires_grad = False

    # Set up token classification head
    self.dropout = nn.Dropout(config.hidden_dropout_prob)
    self.classifier = nn.Linear(config.hidden_size, config.num_labels)
    # Load and initialize weights
    self.init_weights()
  def forward(self, input_ids=None, attention_mask=None, token_type_ids=None,labels=None, **kwargs):
    # Use model body to get encoder representations
    outputs = self.roberta(input_ids, attention_mask=attention_mask,
    token_type_ids=token_type_ids, **kwargs)
    # Apply classifier to encoder representation
    sequence_output = self.dropout(outputs[0])
    logits = self.classifier(sequence_output)
    # Calculate losses
    loss = None
    if labels is not None:
      loss_fct = nn.CrossEntropyLoss()
      loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
    # Return model output object
    return TokenClassifierOutput(loss=loss, logits=logits,
        hidden_states=outputs.hidden_states,
        attentions=outputs.attentions)

## With CRF

In [ ]:
def align_predictions(predictions, label_ids):
  preds = predictions
  batch_size, seq_len = preds.shape
  labels_list, preds_list = [], []
  for batch_idx in range(batch_size):
    example_labels, example_preds = [], []
    for seq_idx in range(seq_len):
      if label_ids[batch_idx, seq_idx] != -100:
        example_labels.append(index2tag[label_ids[batch_idx][seq_idx]])
        example_preds.append(index2tag[preds[batch_idx][seq_idx]])
    labels_list.append(example_labels)
    preds_list.append(example_preds)
  return preds_list, labels_list

def compute_metrics(eval_pred):
  print(eval_pred)
  y_pred, y_true = align_predictions(eval_pred.predictions,
                                     eval_pred.label_ids)
  return {"f1": f1_score(y_true, y_pred)}

In [ ]:
def encode(r):
  X = tokenizer(r['word_process'], 
                truncation=True,
                padding='max_length', 
                max_length=MAX_LEN)
  previous_word_idx = None
  label_ids = []
  label_ids_mask = []
  for word_idx in X.word_ids():
    if word_idx is None :
      label_ids.append(0)
    else:
      label_ids.append(tag2index[r['tag'][word_idx]])
    
    if word_idx is None or word_idx == previous_word_idx:
      label_ids_mask.append(False)
    elif word_idx != previous_word_idx:
      label_ids_mask.append(True)
    previous_word_idx = word_idx
  return X.input_ids, label_ids, label_ids_mask

def preprocess(df):
  df[['tokenized_sentence','tokenized_labels', 'label_ids_mask']] = df.apply(lambda x: encode(x),  axis=1, result_type='expand')
  return df

word_train_data_group = preprocess(word_train_data_group)
word_val_data_group = preprocess(word_val_data_group)
word_test_data_group = preprocess(word_test_data_group)

In [ ]:
class XLMRobertaForTokenClassification(RobertaPreTrainedModel):
  config_class = XLMRobertaConfig
  def __init__(self, config, num_layer=197):
    super().__init__(config)
    self.num_labels = config.num_labels
    # Load model body
    self.roberta = RobertaModel(config, add_pooling_layer=False)
    # Set up token classification head
    self.dropout = nn.Dropout(config.hidden_dropout_prob)
    self.classifier = nn.Linear(config.hidden_size, config.num_labels)
    self.crf = CRF(num_tags=self.num_labels)
    # Load and initialize weights
    self.init_weights()
  def forward(self, input_ids=None, attention_mask=None, token_type_ids=None,labels=None, **kwargs):
    outputs = self.roberta(input_ids=input_ids,
                        token_type_ids=token_type_ids,
                        attention_mask=attention_mask)
    sequence_output = self.dropout(outputs[0])
    emissions = self.classifier(sequence_output)

    if labels is not None:
        log_likelihood = self.crf(emissions, labels)
        sequence_of_tags = torch.Tensor(self.crf.decode(emissions))
        return TokenClassifierOutput(loss=-log_likelihood, logits=torch.Tensor(sequence_of_tags).T)
    else:
        sequence_of_tags = self.crf.decode(emissions)
        return sequence_of_tags

## Run model

In [ ]:
datasets_train_test = DatasetDict({
    "train": Dataset.from_pandas(word_train_data_group[['tokenized_sentence', 'tokenized_labels']].rename(columns={'tokenized_sentence': 'input_ids',
                                                                                  'tokenized_labels': 'labels'})),
    "valid": Dataset.from_pandas(word_val_data_group[['tokenized_sentence', 'tokenized_labels']].rename(columns={'tokenized_sentence': 'input_ids',
                                                                                  'tokenized_labels': 'labels'})),
    "test": Dataset.from_pandas(word_test_data_group[['tokenized_sentence', 'tokenized_labels']].rename(columns={'tokenized_sentence': 'input_ids',
                                                                                  'tokenized_labels': 'labels'}))
    })
datasets_train_test

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels', 'idx'],
        num_rows: 5027
    })
    valid: Dataset({
        features: ['input_ids', 'labels', 'idx'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['input_ids', 'labels', 'idx'],
        num_rows: 3000
    })
})

In [ ]:
num_epochs = 10
batch_size = 8

def model_init():
  return (XLMRobertaForTokenClassification.from_pretrained(pre_trained_name, config=config).to(device))

training_args = TrainingArguments(
      output_dir=pre_trained_name+'_output', log_level="error", num_train_epochs=num_epochs,
      per_device_train_batch_size=batch_size,
      per_device_eval_batch_size=batch_size, evaluation_strategy="epoch",
      logging_steps=len(datasets_train_test["train"]) // batch_size,
       save_steps=1e6, weight_decay=0.01, disable_tqdm=False, push_to_hub=False)

data_collator = DataCollatorForTokenClassification(tokenizer)
trainer = Trainer(model_init=model_init, args=training_args,
        data_collator=data_collator, compute_metrics=compute_metrics,
        train_dataset=datasets_train_test["train"],
        eval_dataset=datasets_train_test["valid"],
        tokenizer=tokenizer)

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [ ]:
#Without CRF
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1
1,0.516700,0.200825,0.793147
2,0.132000,0.163368,0.857644
3,0.085500,0.139034,0.890225
4,0.067900,0.124647,0.897166
5,0.052800,0.134440,0.900896
6,0.040500,0.155354,0.909023
7,0.031900,0.152017,0.913629
8,0.025500,0.152288,0.915509
9,0.019700,0.160086,0.921658
10,0.017800,0.165038,0.921875


TrainOutput(global_step=6290, training_loss=0.09886944247779861, metrics={'train_runtime': 1361.6486, 'train_samples_per_second': 36.918, 'train_steps_per_second': 4.619, 'total_flos': 1642190696678400.0, 'train_loss': 0.09886944247779861, 'epoch': 10.0})

In [ ]:
#With CRF
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchcrf/__init__.py:249: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at ../aten/src/ATen/native/TensorCompare.cpp:413.)
  score = torch.where(mask[i].unsqueeze(1), next_score, score)


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,F1
1,224.038500,82.966591,0.888035
2,59.098800,76.130905,0.910602
3,40.466100,69.364769,0.922891
4,30.668000,76.120110,0.931034
5,25.559800,79.982460,0.939041
6,22.267900,87.241516,0.938337
7,19.718400,106.996284,0.941807
8,17.112500,116.946159,0.943400
9,14.626400,120.375656,0.945312
10,12.170000,122.105522,0.945768


TrainOutput(global_step=6290, training_loss=46.51228413907826, metrics={'train_runtime': 2110.9578, 'train_samples_per_second': 23.814, 'train_steps_per_second': 2.98, 'total_flos': 3284398380595200.0, 'train_loss': 46.51228413907826, 'epoch': 10.0})

In [ ]:
trainer.save_model('xml-r-base-crf')

# Evaluation

## Without CRF

In [ ]:
model = XLMRobertaForTokenClassification.from_pretrained('/content/drive/MyDrive/xml-r-base', config=config)
trainer = Trainer(model=model)
x=trainer.predict(datasets_train_test['test'])

loading weights file /content/drive/MyDrive/xml-r-base/pytorch_model.bin
All model checkpoint weights were used when initializing XLMRobertaForTokenClassification.

All the weights of XLMRobertaForTokenClassification were initialized from the model checkpoint at /content/drive/MyDrive/xml-r-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use XLMRobertaForTokenClassification for predictions without further training.
No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the test set don't have a corresponding argument in `XLMRobertaForTokenClassification.forward` and have been ignored: idx. If idx are not expec

In [ ]:
pred = np.argmax(x.predictions, axis=2)
lab = np.array(datasets_train_test['test']['labels'])
preds_list, labels_list = align_predictions(x.predictions, lab)
print(classification_report(labels_list, preds_list, digits=4))

                     precision    recall  f1-score   support

                AGE     0.9436    0.9701    0.9567       569
               DATE     0.9729    0.9866    0.9797      1639
             GENDER     0.9244    0.9800    0.9514       449
                JOB     0.5813    0.6821    0.6277       173
           LOCATION     0.8938    0.9145    0.9040      4398
               NAME     0.8567    0.9123    0.8836       308
       ORGANIZATION     0.8230    0.8294    0.8262       768
         PATIENT_ID     0.9715    0.9720    0.9717      1962
SYMPTOM_AND_DISEASE     0.7549    0.7830    0.7687      1129
     TRANSPORTATION     0.9300    0.9637    0.9466       193

          micro avg     0.8973    0.9185    0.9078     11588
          macro avg     0.8652    0.8994    0.8816     11588
       weighted avg     0.8985    0.9185    0.9083     11588



## With CRF

In [ ]:
model = XLMRobertaForTokenClassification.from_pretrained('/content/drive/MyDrive/xml-r-base-crf', config=config)
trainer = Trainer(model=model)
x=trainer.predict(datasets_train_test['test'])

The following columns in the test set don't have a corresponding argument in `XLMRobertaForTokenClassification.forward` and have been ignored: idx. If idx are not expected by `XLMRobertaForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 3000
  Batch size = 8
/usr/local/lib/python3.8/dist-packages/torchcrf/__init__.py:249: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at ../aten/src/ATen/native/TensorCompare.cpp:413.)
  score = torch.where(mask[i].unsqueeze(1), next_score, score)


In [ ]:
mask=np.array(word_test_data_group['label_ids_mask'].values.tolist())
lab = np.array(datasets_train_test['test']['labels'])
labs=[]
for ind, l in enumerate(lab):
  labs.append(np.where(mask[ind], l, -100))
labs = np.array(labs).astype('int')

In [ ]:
preds_list, labels_list = align_predictions(x.predictions, labs)
print(classification_report(labels_list, preds_list, digits=4))

                     precision    recall  f1-score   support

                AGE     0.9585    0.9736    0.9660       569
               DATE     0.9771    0.9878    0.9824      1639
             GENDER     0.9245    0.9822    0.9525       449
                JOB     0.6359    0.7168    0.6739       173
           LOCATION     0.9112    0.9150    0.9131      4398
               NAME     0.9211    0.9091    0.9150       308
       ORGANIZATION     0.8032    0.8451    0.8236       768
         PATIENT_ID     0.9686    0.9760    0.9723      1962
SYMPTOM_AND_DISEASE     0.7819    0.7874    0.7846      1129
     TRANSPORTATION     0.9250    0.9585    0.9415       193

          micro avg     0.9090    0.9216    0.9153     11588
          macro avg     0.8807    0.9052    0.8925     11588
       weighted avg     0.9097    0.9216    0.9156     11588

